In [12]:
# Step 1: Load SQLite and create in-memory database
import sqlite3
import pandas as pd

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

In [64]:
cursor.executescript("""
DROP TABLE IF EXISTS Employees;
CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY,
    EmployeeName VARCHAR(100),
    Department VARCHAR(50),
    JobTitle VARCHAR(100),
    Salary DECIMAL(10,2),
    HireDate DATE
);
""")

In [65]:
cursor.executescript("""
INSERT INTO Employees VALUES
(1, 'John Smith', 'Finance', 'Analyst', 95000.00, '2018-03-15'),
(2, 'Ava Miller', 'Finance', 'Accountant', 87000.00, '2019-11-25'),
(3, 'Liam Brown', 'Finance', 'Finance Intern', 87000.00, '2020-09-01'),
(4, 'Lucas White', 'Finance', 'Clerk', 72000.00, '2021-02-18'),
(5, 'Sophia Moore', 'IT', 'Software Engineer', 98000.00, '2017-01-10'),
(6, 'James Taylor', 'IT', 'Data Analyst', 88000.00, '2018-06-14'),
(7, 'Olivia Davis', 'IT', 'Support Engineer', 88000.00, '2021-05-20'),
(8, 'Ethan Harris', 'IT', 'Junior Developer', 75000.00, '2022-03-11'),
(9, 'Emma Johnson', 'HR', 'HR Manager', 84000.00, '2016-09-05'),
(10, 'Noah Wilson', 'HR', 'Recruiter', 70000.00, '2020-02-12'),
(11, 'Grace Lee', 'HR', 'Coordinator', 70000.00, '2021-04-15'),
(12, 'Ella Scott', 'HR', 'Intern', 58000.00, '2022-09-10');
""")

In [66]:
def run_query(title, query):
    print(f"\n🔹 {title}")
    display(pd.read_sql_query(query, conn))

In [67]:
run_query("RANK() – Skips ranks for ties",
"""
SELECT EmployeeName, Salary,
       RANK() OVER (ORDER BY Salary DESC) AS RankPosition
FROM Employees;
""")


🔹 RANK() – Skips ranks for ties


,EmployeeName,Salary,RankPosition
0,Sophia Moore,98000,1
1,John Smith,95000,2
2,James Taylor,88000,3
3,Olivia Davis,88000,3
4,Ava Miller,87000,5
5,Liam Brown,87000,5
6,Emma Johnson,84000,7
7,Ethan Harris,75000,8
8,Lucas White,72000,9
9,Noah Wilson,70000,10


In [68]:
run_query("DENSE_RANK() – No rank gaps for ties",
"""
SELECT EmployeeName, Salary,
       DENSE_RANK() OVER (ORDER BY Salary DESC) AS DenseRank
FROM Employees;
""")


🔹 DENSE_RANK() – No rank gaps for ties


,EmployeeName,Salary,DenseRank
0,Sophia Moore,98000,1
1,John Smith,95000,2
2,James Taylor,88000,3
3,Olivia Davis,88000,3
4,Ava Miller,87000,4
5,Liam Brown,87000,4
6,Emma Johnson,84000,5
7,Ethan Harris,75000,6
8,Lucas White,72000,7
9,Noah Wilson,70000,8


In [69]:
run_query("ROW_NUMBER() – Always unique rank",
"""
SELECT EmployeeName, Salary,
       ROW_NUMBER() OVER (ORDER BY Salary DESC) AS RowNum
FROM Employees;
""")



🔹 ROW_NUMBER() – Always unique rank


,EmployeeName,Salary,RowNum
0,Sophia Moore,98000,1
1,John Smith,95000,2
2,James Taylor,88000,3
3,Olivia Davis,88000,4
4,Ava Miller,87000,5
5,Liam Brown,87000,6
6,Emma Johnson,84000,7
7,Ethan Harris,75000,8
8,Lucas White,72000,9
9,Noah Wilson,70000,10


In [70]:
run_query("LAG() – Previous employee salary",
"""
SELECT EmployeeName, Salary,
       LAG(Salary) OVER (ORDER BY Salary) AS PrevSalary
FROM Employees;
""")


🔹 LAG() – Previous employee salary


,EmployeeName,Salary,PrevSalary
0,Ella Scott,58000,NaN
1,Noah Wilson,70000,58000.0
2,Grace Lee,70000,70000.0
3,Lucas White,72000,70000.0
4,Ethan Harris,75000,72000.0
5,Emma Johnson,84000,75000.0
6,Ava Miller,87000,84000.0
7,Liam Brown,87000,87000.0
8,James Taylor,88000,87000.0
9,Olivia Davis,88000,88000.0


In [71]:
run_query("LEAD() – Next employee salary",
"""
SELECT EmployeeName, Salary,
       LEAD(Salary) OVER (ORDER BY Salary) AS NextSalary
FROM Employees;
""")


🔹 LEAD() – Next employee salary


,EmployeeName,Salary,NextSalary
0,Ella Scott,58000,70000.0
1,Noah Wilson,70000,70000.0
2,Grace Lee,70000,72000.0
3,Lucas White,72000,75000.0
4,Ethan Harris,75000,84000.0
5,Emma Johnson,84000,87000.0
6,Ava Miller,87000,87000.0
7,Liam Brown,87000,88000.0
8,James Taylor,88000,88000.0
9,Olivia Davis,88000,95000.0


In [72]:
run_query("NTILE(4) – Divide into 4 quartiles",
"""
SELECT EmployeeName, Salary,
       NTILE(4) OVER (ORDER BY Salary DESC) AS Quartile
FROM Employees;
""")


🔹 NTILE(4) – Divide into 4 quartiles


,EmployeeName,Salary,Quartile
0,Sophia Moore,98000,1
1,John Smith,95000,1
2,James Taylor,88000,1
3,Olivia Davis,88000,2
4,Ava Miller,87000,2
5,Liam Brown,87000,2
6,Emma Johnson,84000,3
7,Ethan Harris,75000,3
8,Lucas White,72000,3
9,Noah Wilson,70000,4


In [73]:
run_query("FIRST_VALUE() – Top salary per department",
"""
SELECT
    EmployeeName,
    Department,
    Salary,
    FIRST_VALUE(Salary) OVER (PARTITION BY Department ORDER BY Salary DESC) AS TopDeptSalary
FROM Employees;
""")


🔹 FIRST_VALUE() – Top salary per department


,EmployeeName,Department,Salary,TopDeptSalary
0,John Smith,Finance,95000,95000
1,Ava Miller,Finance,87000,95000
2,Liam Brown,Finance,87000,95000
3,Lucas White,Finance,72000,95000
4,Emma Johnson,HR,84000,84000
5,Noah Wilson,HR,70000,84000
6,Grace Lee,HR,70000,84000
7,Ella Scott,HR,58000,84000
8,Sophia Moore,IT,98000,98000
9,James Taylor,IT,88000,98000


In [74]:
run_query("LAST_VALUE() – Lowest salary per department",
"""
SELECT
    EmployeeName,
    Department,
    Salary,
    LAST_VALUE(Salary) OVER (
        PARTITION BY Department
        ORDER BY Salary DESC
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) AS BottomDeptSalary
FROM Employees;
""")


🔹 LAST_VALUE() – Lowest salary per department


,EmployeeName,Department,Salary,BottomDeptSalary
0,John Smith,Finance,95000,72000
1,Ava Miller,Finance,87000,72000
2,Liam Brown,Finance,87000,72000
3,Lucas White,Finance,72000,72000
4,Emma Johnson,HR,84000,58000
5,Noah Wilson,HR,70000,58000
6,Grace Lee,HR,70000,58000
7,Ella Scott,HR,58000,58000
8,Sophia Moore,IT,98000,75000
9,James Taylor,IT,88000,75000
